<a href="https://colab.research.google.com/github/divtosz/projects/blob/main/DevelopmentServerUsingGCandFlask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)
REFERENCE_FILES_PATH = os.path.join(PROJECT_ROOT_DIR, 'references')
os.makedirs(REFERENCE_FILES_PATH, exist_ok=True)

In [13]:
!pip install flask-ngrok

In [14]:
import io
import json

from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)
imagenet_class_index = json.load(open(os.path.join(REFERENCE_FILES_PATH, 
'imagenet_class_index.json')))

model = models.densenet121(pretrained=True)
model.eval()

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]

@app.route("/")
def root():
    return 'Root of Flask WebApp!'

@app.route('/predict')
def predict():
    file_doc = open(os.path.join(IMAGES_PATH, 'wolf.jpg'),'rb')
    img_bytes = file_doc.read()
    class_id, class_name = get_prediction(image_bytes=img_bytes)
    return jsonify({'class_id': class_id, 'class_name': class_name})


In [15]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://be8dd2d747fc.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Feb/2021 10:28:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2021 10:28:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Feb/2021 10:30:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2021 10:30:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2021 10:31:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2021 10:31:32] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2021 10:31:33] "GET /favicon.ico HTTP/1.1" 404 -
